In [7]:
import numpy as np 
import pandas as pd
import re
import string
import sklearn
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Embedding, Dense
from tensorflow.keras import Sequential

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train_set = train['text']
test_set = test['text']
train_targets = train['target']

In [3]:
# Preprocessing
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [8]:
X_train, X_test, y_train, y_test = train_test_split(train_set, train_targets, random_state=0)

In [9]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

In [10]:
train_text[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [11]:
word_list = nltk.word_tokenize(X_train[0])
lemmatized = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
print(word_list, lemmatized)

['Our', 'Deeds', 'are', 'the', 'Reason', 'of', 'this', '#', 'earthquake', 'May', 'ALLAH', 'Forgive', 'us', 'all'] Our Deeds are the Reason of this # earthquake May ALLAH Forgive u all


In [12]:
train_set = []
for i, j in enumerate(train_text):
  word_list = nltk.word_tokenize(train_text[i])
  train_set.append(' '.join([lemmatizer.lemmatize(i) for i in word_list]))

In [ ]:
print(train_set[1], train_text[1])

Forest fire near La Ronge Sask . Canada Forest fire near La Ronge Sask. Canada


In [13]:
counter = CountVectorizer(stop_words = 'english', ngram_range = (2,2))
counter.fit_transform(X_train, y_train)
X_test_bow = counter.transform(X_test)
X_train_bow = counter.transform(X_train)

In [14]:
print(f'The training set has: {X_train_bow.shape[0]:5d} samples and {X_train_bow.shape[1]} features.')
print(f'The test set has:     {X_test_bow.shape[0]:5d} samples and {X_test_bow.shape[1]} features.')

The training set has:  5709 samples and 40647 features.
The test set has:      1904 samples and 40647 features.


In [ ]:
# Simple RNN NN that doesn't work with the data inputs because I'm not great at data visualization.

#layers_simple = [
#    Embedding(20000, 64),
#    SimpleRNN(64, dropout = 0.2, recurrent_dropout = 0.2),
#    Dense(1, activation = 'sigmoid')
#]
#rnn_simple = Sequential(layers_simple)

In [ ]:
#n_epochs = 1

#rnn_simple.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#history_simple = rnn_simple.fit(X_train_bow, y_train,
#                                validation_data=(X_test_bow, y_test),
#                                batch_size=32, epochs=n_epochs)

InvalidArgumentError: ignored

In [15]:
# MultinomialNB that gives good results but is not unsupervised.

for feat_name, train_feat, test_feat in zip(["Bag of Words"],
                                            [X_train_bow],
                                            [X_test_bow]):
 
    mnb = MultinomialNB()
    mnb.fit(train_feat, y_train)
    y_pred = mnb.predict(test_feat)
    print(f"Results for {feat_name}")
    print("-"*80)
    print(classification_report(y_test, y_pred))
    print("-"*80)

Results for Bag of Words
--------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.74      0.88      0.80      1107
           1       0.77      0.56      0.65       797

    accuracy                           0.75      1904
   macro avg       0.75      0.72      0.72      1904
weighted avg       0.75      0.75      0.74      1904

--------------------------------------------------------------------------------
